In [28]:
from bs4 import BeautifulSoup
import pandas as pd

# Read HTML content form file

with open('options.html', 'r') as html_file:
    html_text = html_file.read()
# Parse HTML content
soup = BeautifulSoup(html_text, 'html.parser')

# Find dropdown
dropdown = soup.find('select')  # replace 'select' with the actual tag or class

# Initialize list for data
data = []

In [29]:
# Iterate over optgroups
for optgroup in dropdown.find_all('optgroup'):
    # Extract faculty from optgroup label
    faculty = optgroup['label']

    # Iterate over options within optgroup
    for option in optgroup.find_all('option'):
        # Extract attributes
        name = option.text
        id = option['value']
        # Extract semester from name
        semester = name.split('(')[-1].split('-')[0].strip()
        # Extract credits from name
        credits = name.split('-')[-1].split('créditos')[0].strip()

        # Create dictionary and append to list
        data.append({'name': name, 'credits': credits, 'id': id, 'semester': semester, 'faculty': faculty})

In [30]:
# Data to dataframe
df = pd.DataFrame(data)
df.head()
# Cut id from name column by splitting before "-"
df['name'] = df['name'].str.split('-').str[1]
# Remove semestre from name column by splitting before "("
df['name'] = df['name'].str.split('(').str[0]

In [31]:
df.head()

,name,credits,id,semester,faculty
0,Fundamentos Físicos de la Informática,6,2040004,Segundo semestre,Grado en Ingeniería Informática-Ingeniería de ...
1,Cálculo Infinitesimal y Numérico,6,2040007,Segundo semestre,Grado en Ingeniería Informática-Ingeniería de ...
2,Estadística,6,2040008,Segundo semestre,Grado en Ingeniería Informática-Ingeniería de ...
3,Estructura de Computadores,6,2040009,Segundo semestre,Grado en Ingeniería Informática-Ingeniería de ...
4,Arquitectura de Computadores,6,2040015,Segundo semestre,Grado en Ingeniería Informática-Ingeniería de ...


In [32]:
!pip install openpyxl

In [33]:
# Remove all accents from every column
df = df.apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

In [45]:
# Save to csv

def df_to_excel(df):
    df.to_csv('courses.csv', index=False, sep=';')
    # Create copy and remove all accents
    df_copy = df.copy()
    df_copy = df_copy.apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

    # Save to excel with one sheet per faculty
    with pd.ExcelWriter('courses.xlsx') as writer:
        for faculty in df_copy['faculty'].unique():
            df_copy[df_copy['faculty'] == faculty].to_excel(writer, sheet_name=faculty, index=False, engine='openpyxl', encoding='utf-8')

In [47]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font
from openpyxl.utils.dataframe import dataframe_to_rows

def df_to_excel_advanced(df):
    df.to_csv('courses_1.csv', index=False, sep=';')
    # Create copy and remove all accents
    df_copy = df.copy()
    df_copy = df_copy.apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

    # Save to excel with one sheet per faculty
    with pd.ExcelWriter('courses_1.xlsx') as writer:
        for faculty in df_copy['faculty'].unique():
            df_copy[df_copy['faculty'] == faculty].to_excel(writer, sheet_name=faculty, index=False, engine='openpyxl')

    # Open the Excel file
    wb = load_workbook('courses_1.xlsx')

    # Apply styles
    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
        fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
        font = Font(bold=True)
        for column in ws.columns:
            max_length = 0
            column = [cell for cell in column]
            for cell in column:
                try:  # Necessary to avoid error on empty cells
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length + 2)
            ws.column_dimensions[column[0].column_letter].width = adjusted_width
            column[0].fill = fill
            column[0].font = font

    # Save the styled Excel file
    wb.save('courses_styled.xlsx')

- Visual Computing (VC)
- Software Systems and Engineering (SE)
- Scientific Computing (SC)
- Algorithmic Data Analysis and Machine Learning (AM)
- Algorithmics and Theoretical Computer Science (AT)
- Computer Engineering (CE)

In [35]:
# Assign each module a subject area from this list
# - Visual Computing (VC)
# - Software Systems and Engineering (SE)
# - Scientific Computing (SC)
# - Algorithmic Data Analysis and Machine Learning (AM)
# - Algorithmics and Theoretical Computer Science (AT)
# - Computer Engineering (CE)
subject_areas = [
    'Visual Computing (VC)',
    'Software Systems and Engineering (SE)',
    'Scientific Computing (SC)',
    'Algorithmic Data Analysis and Machine Learning (AM)',
    'Algorithmics and Theoretical Computer Science (AT)',
    'Computer Engineering (CE)'
]

# Add a column to the dataframe with the subject area
df['subject_area'] = df['name'].apply(lambda x: subject_areas[0])
df.head()

,name,credits,id,semester,faculty,subject_area
0,Fundamentos Fisicos de la Informatica,6,2040004,Segundo semestre,Grado en Ingenieria Informatica-Ingenieria de ...,Visual Computing (VC)
1,Calculo Infinitesimal y Numerico,6,2040007,Segundo semestre,Grado en Ingenieria Informatica-Ingenieria de ...,Visual Computing (VC)
2,Estadistica,6,2040008,Segundo semestre,Grado en Ingenieria Informatica-Ingenieria de ...,Visual Computing (VC)
3,Estructura de Computadores,6,2040009,Segundo semestre,Grado en Ingenieria Informatica-Ingenieria de ...,Visual Computing (VC)
4,Arquitectura de Computadores,6,2040015,Segundo semestre,Grado en Ingenieria Informatica-Ingenieria de ...,Visual Computing (VC)


In [36]:
# Call the gpt api to assign each module a subject area from this list
# - Visual Computing (VC)
# - Software Systems and Engineering (SE)
# - Scientific Computing (SC)
# - Algorithmic Data Analysis and Machine Learning (AM)
# - Algorithmics and Theoretical Computer Science (AT)
# - Computer Engineering (CE)



In [37]:
!pip install -U setuptools
!pip install python-dotenv

In [38]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
client = OpenAI(api_key = os.environ['OPEN_API_KEY'], organization=os.environ['ORG_ID'])

In [39]:
def get_completion_from_messages(promt, model='gpt-3.5-turbo-instruct', temperature=0, max_tokens=4):
    response = client.completions.create(
    model=model,
    prompt=promt,
    temperature=temperature, 
    max_tokens=max_tokens,
    )
    completion = response.choices[0].text
    # Remove newline and return characters
    completion = completion.replace('\n', '').replace('\r', '')
    return completion

In [40]:
import csv
import requests
import tqdm

# Get list of modules from dataframe
modules = df['name'].tolist()
# Get string from subject areas list
subject_areas_string = '\n'.join(subject_areas)

# For each module, call the GPT API
for module in tqdm.tqdm(modules):
    promt = f'Given the subject areas "{subject_areas_string}"\nWhich subject area does the module "{module}" belong to? Only respond with the exact name of the subject area.'
    completion = get_completion_from_messages(promt, max_tokens=10)
    print(f"Module: {module} in subject area: {completion}")
    # Assign the subject area to the dataframe
    df.loc[df['name'] == module, 'subject_area'] = completion



  1%|          | 1/143 [00:00<01:44,  1.36it/s]

Module:  Fundamentos Fisicos de la Informatica  in subject area: Scientific Computing (SC)


  1%|▏         | 2/143 [00:01<01:27,  1.62it/s]

Module:  Calculo Infinitesimal y Numerico  in subject area: Scientific Computing (SC)


  2%|▏         | 3/143 [00:01<01:24,  1.66it/s]

...

Module:  Modelado y Analisis de Requisitos en Sistemas de Informacion  in subject area: Software Systems and Engineering (SE)


 99%|█████████▉| 142/143 [00:57<00:00,  2.61it/s]

Module:  Modelos de Computacion y Complejidad  in subject area: Algorithmics and Theoretical Computer Science (AT


100%|██████████| 143/143 [00:58<00:00,  2.46it/s]

Module:  Introduccion a la Ingenieria del Software y los Sistemas de Informacion II  in subject area: Software Systems and Engineering (SE)


In [41]:
df.head()

,name,credits,id,semester,faculty,subject_area
0,Fundamentos Fisicos de la Informatica,6,2040004,Segundo semestre,Grado en Ingenieria Informatica-Ingenieria de ...,Scientific Computing (SC)
1,Calculo Infinitesimal y Numerico,6,2040007,Segundo semestre,Grado en Ingenieria Informatica-Ingenieria de ...,Scientific Computing (SC)
2,Estadistica,6,2040008,Segundo semestre,Grado en Ingenieria Informatica-Ingenieria de ...,Algorithmic Data Analysis and Machine Learning...
3,Estructura de Computadores,6,2040009,Segundo semestre,Grado en Ingenieria Informatica-Ingenieria de ...,Computer Engineering (CE)
4,Arquitectura de Computadores,6,2040015,Segundo semestre,Grado en Ingenieria Informatica-Ingenieria de ...,Computer Engineering (CE)


In [43]:
# head of column subject_area
df['subject_area'].head(20)

0                             Scientific Computing (SC)
1                             Scientific Computing (SC)
2     Algorithmic Data Analysis and Machine Learning...
3                             Computer Engineering (CE)
4                             Computer Engineering (CE)
5     Algorithmics and Theoretical Computer Science (AT
6                             Computer Engineering (CE)
7                             Computer Engineering (CE)
8                 Software Systems and Engineering (SE)
9     Algorithmics and Theoretical Computer Science (AT
10                            Computer Engineering (CE)
11    Algorithmic Data Analysis and Machine Learning...
12    Algorithmic Data Analysis and Machine Learning...
13    Algorithmic Data Analysis and Machine Learning...
14                            Computer Engineering (CE)
15                                Visual Computing (VC)
16                            Computer Engineering (CE)
17                            Computer Engineeri

In [46]:
df_to_excel(df)

In [48]:
df_to_excel_advanced(df)

IndexError: At least one sheet must be visible